This notebook contains pieces of the source code files that have useful functionalities. 
1. Creating the devices
2. Saving frame from one camera in image buffer
3. Displaying a frame from one camera from buffer
4. Streaming video from one camera
5. Saving frame from one camera as PNG
6. Resetting the device settings

##### Important notes: 
- When creating devices, we must destroy them using system.destroy_device()
- Any function associated with BufferFactory that creates a BufferFactory object or a copy of a buffer must destroy the Buffer object to avoid memory leak (since this stores images in memory) - use BufferFactory.destroy(object)

## Imports
Some imports are duplicates from another cell

#### 2. Saving frame from one camera in image buffer

In [ ]:
import time
from arena_api.system import system

#### 3. Displaying a frame from one camera from buffer

In [ ]:
import time

import numpy as np # pip3 install numpy
import cv2  # pip3 install opencv-python
from matplotlib import pyplot as plt # pip3 install matplotlib
# pip3 install pillow
from PIL import Image as PIL_Image
from PIL import ImageTk as PIL_ImageTk
# pip3 install tk / or 'sudo apt-get install python3-tk' for linux
from tkinter import *

from arena_api import enums
from arena_api.system import system
from arena_api.buffer import BufferFactory

#### 4. Streaming video from one camera

In [ ]:
from arena_api.system import system
from arena_api.buffer import *

import ctypes
import numpy as np
import cv2
import time

#### 5. Saving from one camera as PNG

In [ ]:
import time
from datetime import datetime

from arena_api.enums import PixelFormat
from arena_api.__future__.save import Writer
from arena_api.system import system
from arena_api.buffer import BufferFactory

#### 6. Resetting device settings

In [ ]:
import time
from arena_api.system import system

## 1. Creating devices
This step needs to be done before we do anything with the cameras

In [ ]:
"""
This function will let users know that a device is needed and
gives them a chance to connect a device instead of raising an exception
"""
tries = 0
tries_max = 6
sleep_time_secs = 10
while tries < tries_max:  # Wait for device for 60 seconds
    devices = system.create_device() #obtain list of cameras
    if not devices:
        print(
            f'Try {tries+1} of {tries_max}: waiting for {sleep_time_secs}'
            f'secs for a device to be connected!')
        for sec_count in range(sleep_time_secs):
            time.sleep(1)
            print(f'{sec_count + 1 } seconds passed ',
                    '.' * sec_count, end='\r')
        tries += 1
    else:
        break
else:
    raise Exception(f'No device found! Please connect a device and run '
                    f'the example again.')
    
print(devices)

## 2. Saving frame from one camera in image buffer

In [ ]:
TAB1 = "  " #global vars for printing
TAB2 = "    "

In [ ]:
print(f"{TAB1}Enumerate Device")

device = devices[0] #picks one of the devices - sort of at random

# Get stream nodemap to set features before streaming
stream_nodemap = device.tl_stream_nodemap

# Enable stream auto negotiate packet size
stream_nodemap['StreamAutoNegotiatePacketSize'].value = True
# Enable stream packet resend
stream_nodemap['StreamPacketResendEnable'].value = True


In [ ]:
with device.start_stream(): #using "with" we automatically start streaming if all goes well

    buffer = device.get_buffer() #gets image from buffer
    print(f"{TAB2}Acquire Image")

    # Requeue to release buffer memory
    device.requeue_buffer(buffer) #something we should do after getting from buffer
# Clean up
print(f"{TAB1}Clean up Arena")

system.destroy_device()

## 3. Displaying a frame from one camera from buffer

#### Set initial values of attributes

In [ ]:
nodes = device.nodemap.get_node(['Width', 'Height', 'PixelFormat'])
width_initial = nodes['Width'].value
height_initial = nodes['Height'].value
pixel_format_initial = nodes['PixelFormat'].value

In [ ]:
# Get stream nodemap
stream_nodemap = device.tl_stream_nodemap

# Enable stream auto negotiate packet size
stream_nodemap['StreamAutoNegotiatePacketSize'].value = True

# Enable stream packet resend
stream_nodemap['StreamPacketResendEnable'].value = True

# Set width and height to their max values
print('Setting \'Width\' and \'Height\' Nodes value to their max values')
nodes['Width'].value = nodes['Width'].max
nodes['Height'].value = nodes['Height'].max

# Setting pixel format 
new_pixel_format = 'Mono8'
print(f'Setting \'PixelFormat\' to \'{new_pixel_format}\'')
nodes['PixelFormat'].value = new_pixel_format

#### Function that converts the buffer image to BGR8 pixels

In [ ]:
def convert_buffer_to_BGR8(buffer):

    if (buffer.pixel_format == enums.PixelFormat.BGR8):
        return buffer
    print('Converting image buffer pixel format to BGR8 ')
    return BufferFactory.convert(buffer, enums.PixelFormat.BGR8)


#### Turn image data into a numpy array to display

In [ ]:
buffer_BGR8 = None
np_array_reshaped = None

with device.start_stream():
    print(f'Stream started with 1 buffers')
    
    print('\tGet one buffer')
    buffer = device.get_buffer() #grab the image

    # Print some info about the image in the buffer
    print(f'\t\tbuffer received   | '
        f'Width = {buffer.width} pxl, '
        f'Height = {buffer.height} pxl, '
        f'Pixel Format = {buffer.pixel_format.name}')

    # Converting to BGR8 format
    print('\tConverting to BGR8 format')
    buffer_BGR8 = convert_buffer_to_BGR8(buffer)

    # Requeue to release buffer memory
    print('Requeuing device buffer')
    device.requeue_buffer(buffer)

    # Get a copy so it can be used after the buffer is requeued
    print('\tConvert image buffer to a numpy array')
    buffer_bytes_per_pixel = int(len(buffer_BGR8.data)/(buffer_BGR8.width * buffer_BGR8.height))
    np_array = np.asarray(buffer_BGR8.data, dtype=np.uint8) #image data as np array
    np_array_reshaped = np_array.reshape(buffer_BGR8.height, buffer_BGR8.width, buffer_bytes_per_pixel)

#### Display using matplotlib

In [ ]:
np_array_shaped_rgb = cv2.cvtColor(np_array_reshaped, cv2.COLOR_BGR2RGB)
plt.imshow(np_array_shaped_rgb)

#### Display using cv2

In [ ]:
cv2.imshow("window_title", np_array_reshaped)

# wait for user key before closing it
cv2.waitKey(0)

cv2.destroyAllWindows()

#### Display using Tkinter

In [ ]:
print('Creating \'PIL.Image\' instance from Numpy array')
pil_image = PIL_Image.fromarray(np_array_reshaped)

print('Creating a Tkinter readable image from \'PIL.Image\' instance')
root = Tk()
pil_imagetk_photoimage = PIL_ImageTk.PhotoImage(pil_image)

label = Label(root, image=pil_imagetk_photoimage)
label.pack()
root.mainloop()

#### Cleanup and reset initial image values

In [ ]:
BufferFactory.destroy(buffer_BGR8)

In [ ]:
print("Stream stopped")
device.stop_stream()

print("Return nodes to initial values")
nodes['Width'].value = width_initial
nodes['Height'].value = height_initial
nodes['PixelFormat'].value = pixel_format_initial

system.destroy_device()
print('Destroyed all created devices')

## 4. Streaming video from one camera

#### Setup the attibutes

In [ ]:
nodemap = device.nodemap
nodes = nodemap.get_node(['Width', 'Height', 'PixelFormat'])

nodes['Width'].value = 1280
nodes['Height'].value = 720
nodes['PixelFormat'].value = 'RGB8'

num_channels = 3

# Stream nodemap
tl_stream_nodemap = device.tl_stream_nodemap

tl_stream_nodemap["StreamBufferHandlingMode"].value = "NewestOnly"
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True
tl_stream_nodemap['StreamPacketResendEnable'].value = True

#### Stream the video by keep on acquiring the image from the buffer and displaying it using cv2. It also displays a running clock.

In [ ]:
curr_frame_time = 0
prev_frame_time = 0

with device.start_stream():
    """
    Infinitely fetch and display buffer data until esc is pressed
    """
    while True:
        # Used to display FPS on stream
        curr_frame_time = time.time()

        buffer = device.get_buffer()
        """
        Copy buffer and requeue to avoid running out of buffers
        """
        item = BufferFactory.copy(buffer)
        device.requeue_buffer(buffer)

        buffer_bytes_per_pixel = int(len(item.data)/(item.width * item.height))
        """
        Buffer data as cpointers can be accessed using buffer.pbytes
        """
        array = (ctypes.c_ubyte * num_channels * item.width * item.height).from_address(ctypes.addressof(item.pbytes))
        """
        Create a reshaped NumPy array to display using OpenCV
        """
        npndarray = np.ndarray(buffer=array, dtype=np.uint8, shape=(item.height, item.width, buffer_bytes_per_pixel))
        
        fps = str(1/(curr_frame_time - prev_frame_time))
        cv2.putText(npndarray, fps, (7, 70), cv2.FONT_HERSHEY_SIMPLEX, 3, (100, 255, 0), 3, cv2.LINE_AA)
        

        cv2.imshow('Lucid', npndarray)
        """
        Destroy the copied item to prevent memory leaks
        """
        BufferFactory.destroy(item)

        prev_frame_time = curr_frame_time

        """
        Break if esc key is pressed
        """
        key = cv2.waitKey(1)
        if key == 27:
            break
        
    device.stop_stream()
    cv2.destroyAllWindows() 

#### Cleanup

In [ ]:
system.destroy_device()

## 5. Saving frame from one camera as PNG

In [ ]:
TAB1 = "  " #global variables
pixel_format = PixelFormat.BGR8

#### Setup attributes

In [ ]:
"""
Setup stream values
"""
tl_stream_nodemap = device.tl_stream_nodemap
tl_stream_nodemap['StreamAutoNegotiatePacketSize'].value = True
tl_stream_nodemap['StreamPacketResendEnable'].value = True

#### Obtain the image buffer and convert it to a saveable format

In [ ]:
device.start_stream()
buffer = device.get_buffer()

In [ ]:
converted = BufferFactory.convert(buffer, pixel_format)
print(f"{TAB1}Converted image to {pixel_format.name}")

#### Save the image in the /images folder (stored in Jupyter) using a Writer object

In [ ]:
print(f'{TAB1}Prepare Image Writer')
writer = Writer()
writer.pattern = 'images/image_<count>.jpg'


In [ ]:
writer.save(converted)
print(f'{TAB1}Image saved')

#### Cleanup

In [ ]:
BufferFactory.destroy(converted)

device.requeue_buffer(buffer)

In [ ]:
device.stop_stream()

# Destroy Device
system.destroy_device(device)

## 6. Resetting device settings

#### Reset the original settings

In [ ]:
device.nodemap['UserSetSelector'].value = 'Default'
device.nodemap['UserSetLoad'].execute()
print('Device settings has been reset to \'Default\' user set')

#### Cleanup

In [ ]:
# Destroy all created devices. This call is optional and will
# automatically be called for any remaining devices when the system module
# is unloading.
system.destroy_device()
print('Destroyed all created devices')